In [ ]:
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.models import resnet18
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm
import os
from utils import *

# --- Setup ---
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# --- Load pretrained encoder and attach linear classifier ---
encoder = load_pretrained_encoder('./checkpoints_new_backbone/simsiam_encoder.pth', backbone='resnet18')
model = create_linear_classifier(encoder, num_classes=10, freeze_encoder=True).to(device)

# --- EuroSAT Dataset ---
# EuroSAT images are RGB with shape (64, 64), but we'll resize to 224x224
transform = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5]*3, std=[0.5]*3)
])

data_root = './data/EuroSAT/2750'

train_set = datasets.ImageFolder(root=data_root, transform=transform)
num_classes = len(train_set.classes)

# Optionally split dataset into train/test if not already split
from torch.utils.data import random_split
train_size = int(0.8 * len(train_set))
test_size = len(train_set) - train_size
train_set, test_set = random_split(train_set, [train_size, test_size])

train_loader = DataLoader(train_set, batch_size=128, shuffle=True, num_workers=4)
test_loader = DataLoader(test_set, batch_size=128, shuffle=False, num_workers=4)

# --- Training setup ---
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model[-1].parameters(), lr=0.001)

writer = SummaryWriter(log_dir='../runs/linear_eval_eurosat')

# --- Training Loop ---
for epoch in range(100):
    model.train()
    correct, total, total_loss = 0, 0, 0.0

    pbar = tqdm(train_loader, desc=f"Epoch {epoch+1:03d}", leave=False)
    for x, y in pbar:
        x, y = x.to(device), y.to(device)

        logits = model(x)
        loss = criterion(logits, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        preds = logits.argmax(dim=1)
        correct += (preds == y).sum().item()
        total += y.size(0)

        pbar.set_postfix({
            'Loss': f"{loss.item():.4f}",
            'Acc': f"{100 * correct / total:.2f}%"
        })

    acc = 100 * correct / total
    writer.add_scalar('Train/Loss', total_loss / len(train_loader), epoch + 1)
    writer.add_scalar('Train/Accuracy', acc, epoch + 1)

    print(f"Epoch {epoch+1:3d} | Loss: {total_loss:.4f} | Train Acc: {acc:.2f}%")

    # --- Evaluate every 100 epochs ---
    if (epoch + 1) % 10 == 0:
        model.eval()
        test_correct, test_total = 0, 0
        with torch.no_grad():
            for x, y in tqdm(test_loader, desc=f"Eval @ Epoch {epoch+1}", leave=False):
                x, y = x.to(device), y.to(device)
                preds = model(x).argmax(dim=1)
                test_correct += (preds == y).sum().item()
                test_total += y.size(0)

        test_acc = 100 * test_correct / test_total
        writer.add_scalar('Test/Accuracy', test_acc, epoch + 1)
        print(f"Test Accuracy @ Epoch {epoch+1:3d}: {test_acc:.2f}%")

# --- Final Evaluation ---
model.eval()
correct, total = 0, 0
with torch.no_grad():
    for x, y in tqdm(test_loader, desc="Final Evaluation", leave=False):
        x, y = x.to(device), y.to(device)
        preds = model(x).argmax(dim=1)
        correct += (preds == y).sum().item()
        total += y.size(0)

final_test_acc = 100 * correct / total
writer.add_scalar('Test/Accuracy', final_test_acc, 400)
print(f"\nFinal Linear Evaluation Accuracy on EuroSAT: {final_test_acc:.2f}%")

# --- Save Model ---
torch.save(model.state_dict(), 'linear_head_eurosat_final_weights.pth')
print("Final model weights saved as 'linear_head_eurosat_final_weights.pth'")


Epoch   1 | Loss: 166.9954 | Train Acc: 70.62%


Epoch   2 | Loss: 111.1044 | Train Acc: 78.12%


Epoch   3 | Loss: 101.7263 | Train Acc: 79.65%


Epoch   4 | Loss: 97.0383 | Train Acc: 80.24%


Epoch   5 | Loss: 93.8080 | Train Acc: 80.75%


...

Epoch  98 | Loss: 58.8972 | Train Acc: 87.78%


Epoch  99 | Loss: 59.3804 | Train Acc: 87.71%


Epoch 100 | Loss: 59.6722 | Train Acc: 87.53%


Test Accuracy @ Epoch 100: 86.65%



Final Linear Evaluation Accuracy on EuroSAT: 86.65%
Final model weights saved as 'linear_head_eurosat_final_weights.pth'
